In [ ]:
# 인증 키 등록

import os
from dotenv import load_dotenv

load_dotenv()

CAMPING_API_KEY = os.getenv("CAMPING_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(CAMPING_API_KEY)

In [ ]:
# url에 parse하여 적용

from urllib.parse import quote

KeyWord = quote("계곡")
print(KeyWord)

In [3]:
url = f"http://apis.data.go.kr/B551011/GoCamping/searchList?serviceKey={CAMPING_API_KEY}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=AppTest&_type=JSON&keyword={KeyWord}"

In [ ]:
# 공공데이터 API 호출하여 키워드를 포함한 캠핑장 데이터 반환

import requests

def get_url_content(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request Exception: {e}")
        return None
    
result = get_url_content(url)
if result is not None:
    print(result)
else:
    print("Failed to retrieve URL content")

In [ ]:
# json 형태의 데이터 읽어들여서 필요한 정보 추출 (캠핑장 이름 : 간략 설명)
import json

data = json.loads(result)
sites = data['response']['body']['items']['item']

for site in sites:
    print(site['facltNm']+" : "+site['intro'])

In [ ]:
# openAI API로 키워드 추출 질문
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role":"system",
            "content":"다음 질문에서 가장 중요한 키워드 단어 하나만 뽑아줘."
        },
        {
            "role":"user",
            "content": "계곡이 있는 캠핑장을 추천해줘."
        }
    ],
    temperature=0,
    max_tokens=1024,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(response.choices[0].message.content)

In [ ]:

# 캠핑 정보 API 호출하여 데이터 반환
result = get_url_content(url)

# 데이터 형태 JSON 으로 변환 후 목록 추출
import json

data = json.loads(result)
sites = data['response']['body']['items']['item']

context = ''
i = 1
for site in sites:
    context = context + str(i) + ") " + site['facltNm'] + ":" + site['intro'] + "\n"
    i = i + 1

print(context)

In [ ]:
# openAI API로 답하기

import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": f"주어진 문단을 보고 다음 질문에 답해줘: 계곡이 있는 캠핑장을 추천해줘."
    },
    {
      "role": "user",
      "content": context
    }
  ],
  temperature=0,
  max_tokens=1024,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)